In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
import os
import itertools
import warnings
import glob
import sys
import networkx as nx
from pyvis import network as net
import plotly.express as px

In [2]:
def load_pub_for_author(mtid):
    tries = 1
    page = 1
    while True:
        os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?cond=authors.mtid;eq;{mtid}&page={page}&format=json" >wget.out 2>wget.err')
        with open("tmp.json", "r") as f:
            try:
                dta = json.load(f)
                if not os.path.exists(f"journals3/{mtid}{page}.json"):
                    json.dump(dta, open(f"journals3/{mtid}{page}.json", "w", encoding='utf8'), indent=4, ensure_ascii=False)
                if not dta["paging"]["last"]:
                    page += 1
                else:
                    break
            except:
                tries += 1
                print("Error, retrying")
                if tries > 30:
                    return []
staff_flt=pd.read_csv('people_flt.csv')
for mtid in tqdm(staff_flt["MTMT"]):
    load_pub_for_author(mtid)


100%|█████████████████████████████████████████| 124/124 [13:26<00:00,  6.50s/it]


In [4]:
staff_flt=pd.read_csv('people_flt.csv')
df=pd.DataFrame(columns =['publishedYear','name','authors','independentCitingPubCount','types','types2'])
for files in os.listdir('journals3/'):
    dta = json.load(open(f'journals3/{files}'))
    for x in dta["content"]:
        auth=""
        for y in x["authorships"]:
            if "author" in y:
                auth=auth+str(y["author"]["mtid"])+","
            cat=""
            if "category" in x:
                cat=x["category"]["label"]
        a=pd.Series({'publishedYear':x["publishedYear"],'name':x["title"],'authors':auth,'independentCitingPubCount':x["independentCitationCount"],'types':x["otype"],'types2':cat})
        df=pd.concat([df, a.to_frame().T], ignore_index=True)
df=df.drop_duplicates(subset=['name'])
df = df[ (df['types'] != 'Thesis') & (df['types'] != 'Achievement') ]
df

,publishedYear,name,authors,independentCitingPubCount,types,types2
0,1998,The analysis of fundamental notions of linear ...,"10009305,",0,Book,Oktatási
1,1998,Which of the cancellative semigroups are groups?,"10009305,",2,JournalArticle,Tudományos
2,1997,"Scott's conjecture is true, position sensitive...","10009305,",3,JournalArticle,Tudományos
3,1996,Algebra and Automated Deduction,"10009305,",0,JournalArticle,Tudományos
4,1993,The set of types of a finitely generated variety,"10008147,10009305,10007528,",11,JournalArticle,Tudományos
...,...,...,...,...,...,...
5026,2002,Mathematica,"10032461,",0,JournalArticle,Ismeretterjesztő
5027,2002,A faktoriális alsó- és felső becslései,"10032461,",0,JournalArticle,Ismeretterjesztő
5028,2002,The Gradient-Fourier method for nonlinear Neum...,"10032461,",1,JournalArticle,Tudományos
5029,2000,Gradient-Fourier method for nonlinear elliptic...,"10032461,",0,PublicationOther,Tudományos


In [6]:
df = df[ (df['types2'] == 'Tudományos')]
df=df[['publishedYear','name','authors','independentCitingPubCount','types']]

In [7]:
x=df['authors'].str.split(',',expand=True)
x=x.drop([32], axis=1)
x=df.join(x)
staff_flt["MTMT"] = staff_flt["MTMT"].apply(str)
for i in range(32):
    mask = x[i].isin(staff_flt["MTMT"])
    x[i] = x[i].where(mask, None)
for i in range(32):
    y=staff_flt
    pubs=y.rename(columns={"MTMT": i})
    x=pd.merge(x, pubs[[i , "Név"]], how='left',  on=[i])
    x=x.drop([i], axis=1)
    x=x.rename(columns={"Név": i})
x=x.drop(["authors"], axis=1)
x['authors'] = x[x.columns[4:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
x=x[['publishedYear','name','authors','independentCitingPubCount','types']]
x              

,publishedYear,name,authors,independentCitingPubCount,types
0,1998,Which of the cancellative semigroups are groups?,Prőhle Péter,2,JournalArticle
1,1997,"Scott's conjecture is true, position sensitive...",Prőhle Péter,3,JournalArticle
2,1996,Algebra and Automated Deduction,Prőhle Péter,0,JournalArticle
3,1993,The set of types of a finitely generated variety,Prőhle Péter,11,JournalArticle
4,1992,PROBLEMS AND RESULTS IN TAME CONGRUENCE THEORY...,Prőhle Péter,22,JournalArticle
...,...,...,...,...,...
4161,2018,Normal Forms of Endomorphism-Valued Power Series,Szabó Szilárd,0,JournalArticle
4162,2017,Interference and Memory Capacity Limitations,Szabó Szilárd,6,JournalArticle
4163,2004,Abraham A. Ungar: Applications of Hyperbolic G...,Lóczi Lajos,0,PublicationOther
4164,2002,The Gradient-Fourier method for nonlinear Neum...,Lóczi Lajos,1,JournalArticle


In [8]:
x1=x['authors'].str.split(',',expand=True)
x1=x1.join(x)
x1
a=x1[[0,1,'types']]
a=a.rename(columns={0: "auth1"})
a=a.rename(columns={1: "auth2"})
b=x1[[0,2,'types']]
b=b.rename(columns={0: "auth1"})
b=b.rename(columns={2: "auth2"})
c=x1[[0,3,'types']]
c=c.rename(columns={0: "auth1"})
c=c.rename(columns={3: "auth2"})
d=x1[[1,2,'types']]
d=d.rename(columns={1: "auth1"})
d=d.rename(columns={2: "auth2"})
e=x1[[1,3,'types']]
e=e.rename(columns={1: "auth1"})
e=e.rename(columns={3: "auth2"})
h=x1[[2,3,'types']]
h=h.rename(columns={2: "auth1"})
h=h.rename(columns={3: "auth2"})
frames = [a, b, c,d,e,h]
result = pd.concat(frames)
result=result.fillna(0)
result=result.groupby(['auth1', 'auth2'], as_index=False).size()
result.sort_values(by=["size"],ascending=False)


,auth1,auth2,size
0,0,0,11663
198,Simonovits András,0,870
47,Faragó István György,0,623
149,Molnár Lajos Gábor,0,499
115,Kornai András,0,446
...,...,...,...
107,Kiss Krisztina,Kovács Sándor,1
106,Kiss Krisztina,Gyurkovics Éva,1
103,Keliger Dániel,Takács Bálint Máté,1
164,Nagy Attila,Wettl Ferenc,1


In [9]:
result.to_csv("publication.csv", index=False, float_format='%.4g')


# citation

In [2]:
staff_flt=pd.read_csv('people_flt.csv')
df=pd.DataFrame(columns =['from','to','name1','name2'])
for files in tqdm(os.listdir('journals3/')):
    dta = json.load(open(f'journals3/{files}'))
    for x in dta["content"]:
        auth=""
        name1=x["title"]
        for y in x["authorships"]:
            if "author" in y:
                auth=auth+str(y["author"]["mtid"])+","
        page = 1
        cimtid=x["mtid"]
        while True:
            os.system(f'wget -O tmp.json "https://m2.mtmt.hu/api/publication?sort=publishedYear,desc&sort=firstAuthor&sort=title&size=100&cond=published;eq;true&cond=citations.related;eq;{cimtid}&cond=citations.published;eq;true&format=json" >wget.out 2>wget.err')
            with open("tmp.json", "r") as f:
                try:
                    dta2 = json.load(f)
                    for z in dta2["content"]:
                        name2=z["title"]
                        auth2=""
                        for w in z["authorships"]:
                            if "author" in w:
                                auth2=auth2+str(w["author"]["mtid"])+","
                        a=pd.Series({'from':auth2,'to':auth,'name1':name1,'name2':name2})
                        df=pd.concat([df, a.to_frame().T], ignore_index=True)        
                    if not dta2["paging"]["last"]:
                        page += 1
                    else:
                        break
                except:
                        print("Error, retrying")
                        break
        
df

  2%|▉                                     | 13/563 [33:48<45:59:07, 300.99s/it]

Error, retrying


100%|███████████████████████████████████████| 563/563 [3:09:05<00:00, 20.15s/it]


,from,to,name1,name2
0,"2050687,","10000407,",Norm-graphs: variations and applications,An upper bound on Zarankiewicz' problem
1,"2050687,","10000407,",Norm-graphs: variations and applications,New asymptotics for bipartite Turán numbers
2,"10000407,","10000407,",Norm-graphs: variations and applications,Norm-graphs and bipartite Turán numbers
3,"10039408,10000936,","10000407,",Norm-graphs: variations and applications,On a problem of graph theory
4,"10000936,","10000407,",Norm-graphs: variations and applications,On a problem of Zarankiewicz
...,...,...,...,...
8578,"10004964,10000168,","10000168,10004964,",Variable preconditioning via quasi-Newton meth...,The gradient-finite element method for ellipti...
8579,"10000168,","10000168,10004964,",Variable preconditioning via quasi-Newton meth...,Sobolev space preconditioning of strongly nonl...
8580,"10004964,10000168,","10004964,10000168,",Sobolev space preconditioning for Newton's met...,The gradient-finite element method for ellipti...
8581,"10032461,","10004964,10000168,",Numerical solution of nonlinear elliptic probl...,The Gradient-Fourier method for nonlinear Neum...


In [3]:
df=df.drop_duplicates(subset=['name1', 'name2'])
df=df[['from', 'to']]
df=df.reset_index()

In [4]:
y=df[['from', 'to']]
y

,from,to
0,"2050687,","10000407,"
1,"2050687,","10000407,"
2,"10000407,","10000407,"
3,"10039408,10000936,","10000407,"
4,"10000936,","10000407,"
...,...,...
7554,"10000966,10019030,","10007872,"
7555,"10007845,10012217,","10007872,"
7556,"10007872,10006310,","10007872,"
7557,"10007872,","10007872,"


In [5]:
df=df[['from', 'to']]
x=df['from'].str.split(',',expand=True)
x=x.drop([31], axis=1)
x=df.join(x)
staff_flt["MTMT"] = staff_flt["MTMT"].apply(str)
for i in range(31):
    mask = x[i].isin(staff_flt["MTMT"])
    x[i] = x[i].where(mask, None)
for i in range(31):
    y=staff_flt
    pubs=y.rename(columns={"MTMT": i})
    
    x=pd.merge(x, pubs[[i , "Név"]], how='left',  on=[i])
    x=x.drop([i], axis=1)
    x=x.rename(columns={"Név": i})
x=x.drop(["from"], axis=1)
x['from'] = x[x.columns[1:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
x=x[['to','from']]
index1 = x[ (x['from'] == "") ].index
x.drop(index1 , inplace=True)
x1=x['to'].str.split(',',expand=True)
x1=x1.drop([8], axis=1)
x=x1.join(x)
for i in range(8):
    mask = x[i].isin(staff_flt["MTMT"])
    x[i] = x[i].where(mask, None)
for i in range(8):
    y=staff_flt
    pubs=y.rename(columns={"MTMT": i})
    
    x=pd.merge(x, pubs[[i , "Név"]], how='left',  on=[i])
    x=x.drop([i], axis=1)
    x=x.rename(columns={"Név": i})
x=x.drop(["to"], axis=1)
x['to'] = x[x.columns[1:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
x=x[['to','from']]
index1 = x[ (x['to'] == "") ].index
x.drop(index1 , inplace=True)
x

,to,from
0,Rónyai Lajos,Rónyai Lajos
1,Simonovits András,Simonovits András
2,Simonovits András,Simonovits András
3,Simonovits András,Simonovits András
4,Faragó István György,Faragó István György
...,...,...
4585,Kroó András,Kroó András
4586,Kroó András,Kroó András
4587,Kroó András,Kroó András
4588,Kroó András,Kroó András


In [6]:
x1=x['from'].str.split(',',expand=True)
x1=x1.rename(columns={0: "from1"})
x1=x1.rename(columns={1: "from2"})
x1=x1.rename(columns={2: "from3"})
x2=x['to'].str.split(',',expand=True)
x2=x2.rename(columns={0: "to1"})
x2=x2.rename(columns={1: "to2"})
x2=x2.rename(columns={2: "to3"})
y=x1.join(x2)
a=y[['to1','from1']]
a=a.rename(columns={'to1': "to"})
a=a.rename(columns={'from1': "from"})
b=y[['to2','from1']]
b=b.rename(columns={'to2': "to"})
b=b.rename(columns={'from1': "from"})
c=y[['to3','from1']]
c=c.rename(columns={'to3': "to"})
c=c.rename(columns={'from1': "from"})
d=y[['to1','from2']]
d=d.rename(columns={'to1': "to"})
d=d.rename(columns={'from2': "from"})
e=y[['to2','from2']]
e=e.rename(columns={'to2': "to"})
e=e.rename(columns={'from2': "from"})
f=y[['to3','from2']]
f=f.rename(columns={'to3': "to"})
f=f.rename(columns={'from2': "from"})
g=y[['to1','from3']]
g=g.rename(columns={'to1': "to"})
g=g.rename(columns={'from3': "from"})
h=y[['to2','from3']]
h=h.rename(columns={'to2': "to"})
h=h.rename(columns={'from3': "from"})
i=y[['to3','from3']]
i=i.rename(columns={'to3': "to"})
i=i.rename(columns={'from3': "from"})
frames = [a, b, c,d,e,f,g,h,i]
result = pd.concat(frames)
index1 = result[ (result['to'] == None) ].index
result=result.dropna()

result=result.rename(columns={'to': "target"})
result=result.rename(columns={'from': "source"})
result=result.groupby(['source', 'target'], as_index=False).size()
result.sort_values(by=["size"],ascending=False)
result


,source,target,size
0,Andai Attila,Andai Attila,7
1,Andai Attila,Lovas Attila,2
2,Andai Attila,Szabó Sándor,1
3,Babcsányi István,Babcsányi István,50
4,Babcsányi István,Nagy Attila,17
...,...,...,...
316,Weiner Mihály,Weiner Mihály,52
317,Wettl Ferenc,Babcsányi István,2
318,Wettl Ferenc,Nagy Attila,2
319,Wettl Ferenc,Tóth János,1


In [7]:
result.to_csv("citation_graph.csv", index=False, float_format='%.4g')